# Animals-10 Data Exploration
The [Animals-10 Dataset](https://www.kaggle.com/datasets/alessiocorrado99/animals10?resource=download) is made by Corrado Alessio. There are a multitude of goals that I want to acheive in this exploration. These include:
- Explore Dataset and its characteristics
- Identify issues
- Develop solutions
- Preprocess for image classification training

# Exploring the Dataset
To get a feel of what the dataset is about, the first step would be to read what the author has outlined for us:

> Hello everyone!
> This is the dataset I have used for my matriculation thesis.
> It contains about 28K medium quality animal images belonging to 10 categories: dog, cat, horse, spyder, butterfly, chicken, sheep, cow, squirrel, elephant.
> I have used it to test different image recognition networks: from homemade CNNs (~80% accuracy) to Google Inception (98%). It could simulate a smart gallery for a researcher (like a biologist).
> All the images have been collected from "google images" and have been checked by human. There is some erroneous data to simulate real conditions (eg. images taken by users of your app).
> The main directory is divided into folders, one for each category. Image count for each category varies from 2K to 5 K units.".

As we can see here he mentions that the dataset includes 10 categories:`dog`, `cat`, `horse`, `spyder`, `butterfly`, `chicken`, `sheep`, `cow`, `squirrel`, and `elephant`. We should confirm this as one of the key concepts I have learned is to not trust everything until I am able to verify the datapoint my self. So lets read in the data and print out the different class names found by going through the folder names.

The data zip file can be downloaded from [here](https://www.kaggle.com/datasets/alessiocorrado99/animals10?resource=download).